In [1]:
import pandas as pd
import numpy as np
import json 
import csv
import urllib.request

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [2]:
l=range(10)

out_file_name = "data_int.txt"

with open(out_file_name, 'w') as outfile:
    for n in l:
        outfile.write(str(n) + '\n')
    
!type "data_int.txt" #I use type instead of cat because I'm on window

m = np.empty([5, 5], dtype = float)

out_file_name = "data_float.txt"

with open(out_file_name, 'w') as outfile:
    for line in m:
        for n in line:
            outfile.write(str(n) + ',')
        outfile.write(str(n) + '\n')
        
!type "data_float.txt"

float_csv = pd.read_csv("data_float.txt")
  
float_csv.to_csv('data_float.csv', 
                  index = None)
 

0
1
2
3
4
5
6
7
8
9



data_int.txt


Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: #I.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: use.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: type.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: instead.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: of.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: cat.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: because.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: I'm.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: on.
Le fichier sp‚cifi‚ est introuvable.
Une erreur s'est produite lors du traitement deÿ: window.


6.230420704259778e-307,4.672967463650385e-307,1.6912109553714024e-306,9.346091108988578e-307,1.3351101828069307e-306,1.3351101828069307e-306
1.3351196894206007e-306,6.23037996370202e-307,6.230539538199959e-307,9.346097901447407e-307,8.455939341425617e-307,8.455939341425617e-307
9.346009625593585e-307,7.566011651748551e-307,1.6911878680364725e-306,1.1126102664124752e-306,1.780184032301065e-306,1.780184032301065e-306
1.1126156989289277e-306,8.455593030780116e-307,8.066130403919971e-308,6.898102438071315e-307,1.2238755034223467e-307,1.2238755034223467e-307
2.2252259641187545e-306,8.344239171960076e-308,9.791071931451263e-307,8.455280672999678e-307,3.2055295e-317,3.2055295e-317


2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [3]:
data = json.load(open('user_data.json'))
data_filtered=[x for x in data if x["CreditCardType"]=="American Express"]


with open('data_file.csv', 'w') as data_csv:
 
    # create the csv writer object
    csv_writer = csv.writer(data_csv)

    # Counter variable used for writing
    # headers to the CSV file
    count = 0

    for ppl in data_filtered:
        if count == 0:
 
            # Writing headers of CSV file
            header = ppl.keys()
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(ppl.values())
 
!type "data_file.csv"

ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType

2,Investment  Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express

12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express

28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express

39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express

57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express

62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express

68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express

74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express

81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express

92,Staffing Consultant,Maia_Stark2797@jiman.org,Maia

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [4]:
data=pd.read_csv("mushrooms_categorized.csv")
print(data)
mean_class=data.groupby('class').mean()
mean_class.to_json("mean_mushroom.json")
jsondoc = pd.read_json("mean_mushroom.json")
jsondoc
          

      class  cap-shape  cap-surface  cap-color  bruises  odor  \
0         1          5            2          4        1     6   
1         0          5            2          9        1     0   
2         0          0            2          8        1     3   
3         1          5            3          8        1     6   
4         0          5            2          3        0     5   
...     ...        ...          ...        ...      ...   ...   
8119      0          3            2          4        0     5   
8120      0          5            2          4        0     5   
8121      0          2            2          4        0     5   
8122      1          3            3          4        0     8   
8123      0          5            2          4        0     5   

      gill-attachment  gill-spacing  gill-size  gill-color  ...  \
0                   1             0          1           4  ...   
1                   1             0          0           4  ...   
2                 

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [11]:
with open('credit_card.dat','rb') as file:
    file_content = file.read()
    file_content=file_content.replace(b'\n',b'\r\n') #because I'm on window
    word_size = 6
    current_card=""
    cards=[]
    for i in range(0, len(file_content)-6, word_size):
        n=file_content[i:i+word_size]
        nchar=chr(int(n,2))
        if nchar=="\n":
            cards.append(current_card)
            current_card=""
        else :
            current_card+=(nchar)

print(cards)

['7648 5673 3775 2271', '3257 8247 3354 2266', '2722 0001 4011 6652', '0661 3063 3742 3150', '0432 1608 1462 4742', '5827 2027 8785 7303', '5774 8528 2087 1117', '8140 1210 6352 2845', '5764 1133 7301 7100', '6456 1737 4126 6726', '1228 8631 7382 0000', '7051 0160 5374 3166', '0618 3587 1630 6376', '1545 5454 7444 5636', '6735 3116 3202 6834', '7287 5011 1547 8413', '7033 2607 3328 4200', '2568 5244 1874 5024', '1684 2253 7570 7118', '0672 2576 0575 6631', '6332 8353 8787 1340', '1813 3361 1175 4211', '2477 6450 8840 2368', '5512 3505 2563 1326', '3083 7882 0621 0025', '4521 5148 8045 0334', '7563 3654 8713 5787', '8324 2664 0476 5561', '0565 2504 7168 3510', '5107 5507 1767 0738', '2462 1821 2448 1443', '2788 0638 6861 6554', '5851 5873 5474 0547', '0670 1004 4013 2655', '5874 5506 3048 0806', '2805 5401 8462 1260', '5083 8406 6310 1862', '1076 1445 3013 2266', '8440 4804 4844 5277', '4758 6141 0686 1387', '7586 0675 0315 2568', '2544 1258 7432 5165', '3474 5023 4434 5626', '1410 0270

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3